<a href="https://colab.research.google.com/github/bannis14/RFM_Segmentation/blob/main/Recency_Frequency_Monetary_Segmentation_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
%cd /content/drive/MyDrive/Business Analyst course/Portfolio

/content/drive/MyDrive/Business Analyst course/Portfolio


In [14]:
# Libraries
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt

In [15]:
## Data
df = pd.read_csv('customer_data.csv').dropna()
df.head()

,customer_id,revenue,most_recent_visit,number_of_orders,recency_days
0,22086,777,5/14/2006,9,232
1,2290,1555,9/8/2006,16,115
2,26377,336,11/19/2006,5,43
3,24650,1189,10/29/2006,12,64
4,12883,1229,12/9/2006,12,23


In [16]:
# Prepare monetary basket
df['monetary'] = df['revenue'] / df['number_of_orders']
df.head()

,customer_id,revenue,most_recent_visit,number_of_orders,recency_days,monetary
0,22086,777,5/14/2006,9,232,86.333333
1,2290,1555,9/8/2006,16,115,97.187500
2,26377,336,11/19/2006,5,43,67.200000
3,24650,1189,10/29/2006,12,64,99.083333
4,12883,1229,12/9/2006,12,23,102.416667


In [17]:
# Change the variable names
df = df.rename(columns={'recency_days': 'recency', 'number_of_orders': 'frequency'})
df.head()

df = df.drop(columns=['revenue', 'most_recent_visit'])
df.head()

df['F'] = pd.qcut(df['frequency'], q=3, labels=range(1, 4, 1))
df['M'] = pd.qcut(df['monetary'], q=3, labels=range(1, 4, 1))
df['R'] = pd.qcut(df['recency'], q=3, labels=range(3, 0, -1))

df.head()

df['RFM'] = df[['R', 'F', 'M']].sum(axis=1)
df.head()

,customer_id,frequency,recency,monetary,F,M,R,RFM
0,22086,9,232,86.333333,2,1,1,4
1,2290,16,115,97.187500,3,2,2,7
2,26377,5,43,67.200000,1,1,3,5
3,24650,12,64,99.083333,3,2,3,8
4,12883,12,23,102.416667,3,2,3,8


In [20]:
# Create RFM function
def rfm_segment(df):
    if df['RFM'] >= 8:
        return 'Superstar'
    elif ((df['RFM'] >= 5) and (df['RFM'] < 8)):
        return 'High Potential'
    else:
        return 'Low Relevance'

df['RFM Level'] = df.apply(rfm_segment, axis=1)
df.head()

result_summary = df.groupby('RFM Level').agg({
    'recency': 'mean',
    'frequency': 'mean',
    'monetary': ['mean', 'count']
}).round(1)

result_summary

recency frequency monetary       
                  mean      mean     mean  count
RFM Level                                       
High Potential   171.8       9.8     97.0  26445
Low Relevance    306.6       7.1     78.5   7179
Superstar         80.1      12.8    108.3   6375